### Combined clustering

In [1]:
#importing necessary libraries
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import os.path
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift, AffinityPropagation
from sklearn.mixture import GaussianMixture
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.simplefilter('ignore')

In [2]:
df=pd.read_csv('/content/physical.csv')
df.head()

,No,Block No,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e
0,1,2,2.29,2.14,2.29,2287.0,2143.0,2191.0,2.09,6.57,14.8,0.17
1,2,5A,2.25,NaN,NaN,2253.0,NaN,NaN,2.10,8.37,NaN,NaN
2,3,5B,2.27,2.15,2.28,2269.0,2152.0,2279.0,2.12,7.37,12.7,0.15
3,4,5C,2.24,2.14,2.24,2242.0,2144.0,2243.0,2.12,5.99,9.8,0.11
4,5,5D-A,2.23,2.05,2.22,2233.0,2052.0,2218.0,2.06,8.93,16.6,0.20


In [3]:
df.shape

(150, 12)

In [4]:
df1=pd.read_csv('/content/Mechanical properties.csv', encoding='ISO-8859-1')
df1.head()

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%)
0,"2 (0,1,2)",NaN,0.65,5.77,21.0,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN
1,"5A (1,1,0)",13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN
2,"5B (3,3,3)",9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN
3,"5C (4,2,3)",14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN
4,"5D-A (2,1,2)",11.66,0.99,2.42,13.0,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN


In [5]:
df1.shape

(156, 18)

### Deleting 6 rows using list

*   Physical property has 150 rows
*   Mechanical property has 156 rows.
*   To avoid conflict, using GAD location i deleted the unused rows in mechanical property.
*   Finally i have equal no of rows in both physical and mechanical property.





In [6]:
# Remove the brackets and their contents from the first column
df1['Sample Label(Number of cores for UCS, PLI, ITS)'] = df1['Sample Label(Number of cores for UCS, PLI, ITS)'].str.replace(r'\(\d+,\d+,\d+\)', '')
df1.head()

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%)
0,2,NaN,0.65,5.77,21.0,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN
4,5D-A,11.66,0.99,2.42,13.0,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN


In [7]:
# Convert the "Code" column into a list
l2 = df1['Sample Label(Number of cores for UCS, PLI, ITS)'].tolist()
l2

['2   ',
 '5A ',
 '5B ',
 '5C  ',
 '5D-A  ',
 '5D-B  ',
 '5D-C  ',
 '6   ',
 '7  ',
 '8A ',
 '8B ',
 '8C ',
 '8D ',
 '8E ',
 '8F ',
 '9A ',
 '9B ',
 '9C ',
 '10A ',
 '11A-1 ',
 '11A-2 ',
 '11A-3 ',
 '11A-4 ',
 '11A-5 ',
 '11A-6 ',
 '11A-7 ',
 '11A-8 ',
 '11A-9 ',
 '11A-10 ',
 '11A-11 ',
 '11A-12 ',
 '11A-13 ',
 '11A-14 ',
 '11A-15 ',
 '11A-16 ',
 '11A-17 ',
 '11A-18 ',
 '11A-19 ',
 '11A-20 ',
 '11B-1 ',
 '11B-2 ',
 '11B-3 ',
 '11B-4 ',
 '11B-5 ',
 '11B-6 ',
 '11B-7 ',
 '11B-8 ',
 '11B-9 ',
 '11B-10 ',
 '11B-11 ',
 '11B-12 ',
 '11B-13 ',
 '11B-14 ',
 '11B-15 ',
 '11B-16 ',
 '11B-17 ',
 '11B-18 ',
 '11B-19 ',
 '11B-20 ',
 '12B ',
 '12B-1',
 '12B-2 ',
 '12B-3 ',
 '12B-4 ',
 '12B-5 ',
 '12B-6 ',
 '12B-7 ',
 '12B-8 ',
 '12A-1 ',
 '12A-2 ',
 '12A-3 ',
 '12--1 ',
 '12--2 ',
 '13A-1 ',
 '13A-2 ',
 '13A-3 ',
 '13A-4 ',
 '13A-5 ',
 '13B-1 ',
 '13B-2 ',
 '13-1 ',
 '13-2 ',
 '14A-1 ',
 '14A-2 ',
 '14B-1 ',
 '14B-2 ',
 '14B-3 ',
 '14-1 ',
 '14-2 ',
 '14-3 ',
 '14-4 ',
 '14-5 ',
 '14-6 ',
 '20-1 ',


In [8]:
# Remove unnecessary spaces in each element of the list
l3 = [item.strip() for item in l2]
l3

['2',
 '5A',
 '5B',
 '5C',
 '5D-A',
 '5D-B',
 '5D-C',
 '6',
 '7',
 '8A',
 '8B',
 '8C',
 '8D',
 '8E',
 '8F',
 '9A',
 '9B',
 '9C',
 '10A',
 '11A-1',
 '11A-2',
 '11A-3',
 '11A-4',
 '11A-5',
 '11A-6',
 '11A-7',
 '11A-8',
 '11A-9',
 '11A-10',
 '11A-11',
 '11A-12',
 '11A-13',
 '11A-14',
 '11A-15',
 '11A-16',
 '11A-17',
 '11A-18',
 '11A-19',
 '11A-20',
 '11B-1',
 '11B-2',
 '11B-3',
 '11B-4',
 '11B-5',
 '11B-6',
 '11B-7',
 '11B-8',
 '11B-9',
 '11B-10',
 '11B-11',
 '11B-12',
 '11B-13',
 '11B-14',
 '11B-15',
 '11B-16',
 '11B-17',
 '11B-18',
 '11B-19',
 '11B-20',
 '12B',
 '12B-1',
 '12B-2',
 '12B-3',
 '12B-4',
 '12B-5',
 '12B-6',
 '12B-7',
 '12B-8',
 '12A-1',
 '12A-2',
 '12A-3',
 '12--1',
 '12--2',
 '13A-1',
 '13A-2',
 '13A-3',
 '13A-4',
 '13A-5',
 '13B-1',
 '13B-2',
 '13-1',
 '13-2',
 '14A-1',
 '14A-2',
 '14B-1',
 '14B-2',
 '14B-3',
 '14-1',
 '14-2',
 '14-3',
 '14-4',
 '14-5',
 '14-6',
 '20-1',
 '20-2',
 '21-1',
 '21-2',
 '21-3',
 '22-1',
 '22-2',
 '22-3',
 '22-4',
 '23-1',
 '23-2',
 '23-3',
 '2

In [9]:
# Convert the "Code" column into a list
l1 = df['    Block     No'].tolist()
l1

['2',
 '5A',
 '5B',
 '5C',
 '5D-A',
 '5D-B',
 '5D-C',
 '6',
 '7',
 '8A',
 '8B',
 '8C',
 '8D',
 '8E',
 '8F',
 '9A',
 '9B',
 '9C',
 '10A',
 '11A-1',
 '11A-2',
 '11A-3',
 '11A-4',
 '11A-5',
 '11A-6',
 '11A-7',
 '11A-8',
 '11A-9',
 '11A-10',
 '11A-11',
 '11A-12',
 '11A-13',
 '11A-14',
 '11A-15',
 '11A-16',
 '11A-17',
 '11A-18',
 '11A-19',
 '11A-20',
 '11B-1',
 '11B-2',
 '11B-3',
 '11B-4',
 '11B-5',
 '11B-6',
 '11B-7',
 '11B-8',
 '11B-9',
 '11B-10',
 '11B-11',
 '11B-12',
 '11B-13',
 '11B-14',
 '11B-15',
 '11B-16',
 '11B-17',
 '11B-18',
 '11B-19',
 '11B-20',
 '12B',
 '12B-1',
 '12B-2',
 '12B-3',
 '12B-4',
 '12B-5',
 '12B-6',
 '12B-7',
 '12B-8',
 '12A-1',
 '12A-2',
 '12A-3',
 '12--1',
 '12--2',
 '13A-1',
 '13A-2',
 '13A-3',
 '13A-4',
 '13A-5',
 '13B-1',
 '13B-2',
 '13-1',
 '13-2',
 '14A-1',
 '14A-2',
 '14B-1',
 '14B-2',
 '14B-3',
 '14-1',
 '14-2',
 '14-3',
 '14-4',
 '14-5',
 '14-6',
 '20-1',
 '20-2',
 '21-1',
 '21-2',
 '21-3',
 '22-1',
 '22-2',
 '22-3',
 '22-4',
 '23-1',
 '23-2',
 '23-3',
 '2

In [10]:
# Find elements in list1 but not in list2
unique_elements_in_list2 = list(set(l3) - set(l1))

In [11]:
# Display the results
print("six unique element that are not present in physical property:", unique_elements_in_list2)

six unique element that are not present in physical property: ['40-3', '40-4', '28-2', '28-1', '26-1', '32-6']


In [12]:
d = df1.copy()
d.head()

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%)
0,2,NaN,0.65,5.77,21.0,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN
4,5D-A,11.66,0.99,2.42,13.0,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN


In [13]:
# Blocks to eliminate
blocks_to_eliminate = ['32-6', '28-2', '26-1', '40-3', '40-4', '28-1']

In [14]:
# Filter out rows with specified 'Block No'
d_filter = d[~d['Sample Label(Number of cores for UCS, PLI, ITS)'].str.strip().isin(blocks_to_eliminate)]
d_filter

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%)
0,2,NaN,0.65,5.77,21.00,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN
4,5D-A,11.66,0.99,2.42,13.00,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,37-4,12.28,1.72,2.97,23.75,39.90,NaN,NaN,5782,2926,73.9,65.1,56.3,48.0,63.4,54.5,49.7,43.4
150,39-1,28.15,1.16,2.86,46.00,41.38,NaN,NaN,5656,2874,87.8,80.2,71.8,65.7,NaN,NaN,NaN,NaN
151,39-2,15.83,0.45,2.49,44.00,34.77,NaN,NaN,4056,2079,86.4,79.3,73.3,67.7,NaN,NaN,NaN,NaN
152,40-1,5.60,0.54,1.07,22.00,16.80,3504.0,1800.0,4476,2052,84.8,76.9,68.6,61.2,NaN,NaN,NaN,NaN


In [15]:
# Reset the index without keeping the old index
d_filter = d_filter.reset_index(drop=True)
d_filter

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%)
0,2,NaN,0.65,5.77,21.00,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN
4,5D-A,11.66,0.99,2.42,13.00,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,37-4,12.28,1.72,2.97,23.75,39.90,NaN,NaN,5782,2926,73.9,65.1,56.3,48.0,63.4,54.5,49.7,43.4
146,39-1,28.15,1.16,2.86,46.00,41.38,NaN,NaN,5656,2874,87.8,80.2,71.8,65.7,NaN,NaN,NaN,NaN
147,39-2,15.83,0.45,2.49,44.00,34.77,NaN,NaN,4056,2079,86.4,79.3,73.3,67.7,NaN,NaN,NaN,NaN
148,40-1,5.60,0.54,1.07,22.00,16.80,3504.0,1800.0,4476,2052,84.8,76.9,68.6,61.2,NaN,NaN,NaN,NaN


In [16]:
# Extracting the part before the hyphen
d_filter['GAD'] = d_filter['Sample Label(Number of cores for UCS, PLI, ITS)'].str.split('-', n=1).str[0]

# Displaying the DataFrame with the extracted values
d_filter.head()

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%),GAD
0,2,NaN,0.65,5.77,21.0,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN,2
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN,5A
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN,5B
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN,5C
4,5D-A,11.66,0.99,2.42,13.0,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN,5D


In [17]:
# Removing alphabets from the column
d_filter['GAD'] = d_filter['GAD'].str.replace('[^0-9]', '', regex=True)
d_filter.head()

,"Sample Label(Number of cores for UCS, PLI, ITS)",UCS mean,PLI,ITS mean,SHV (mean),SHV (S) (mean),SV (m/s) mean (B)Vp,SV (m/s) mean (B)Vs,SV (m/s) mean (C)Vp,SV (m/s) mean (C)Vs,SDI (D) Id1 (%),SDI (D) Id2 (%),SDI (D) Id3 (%),SDI (D) Id4 (%),SDI (S) Id1 (%),SDI (S) Id2 (%),SDI (S) Id3 (%),SDI (S) Id4 (%),GAD
0,2,NaN,0.65,5.77,21.0,NaN,NaN,NaN,6548,3143,87.9,76.2,68.8,60.7,NaN,NaN,NaN,NaN,2
1,5A,13.07,0.69,NaN,NaN,NaN,NaN,NaN,5889,3088,81.1,66.9,58.6,50.9,NaN,NaN,NaN,NaN,5
2,5B,9.69,1.03,2.36,NaN,NaN,NaN,NaN,6007,3081,79.9,64.0,56.4,47.3,NaN,NaN,NaN,NaN,5
3,5C,14.61,0.97,1.83,NaN,NaN,NaN,NaN,6029,3049,73.2,59.1,52.5,46.4,NaN,NaN,NaN,NaN,5
4,5D-A,11.66,0.99,2.42,13.0,NaN,NaN,NaN,6327,3215,71.1,60.5,54.8,48.3,NaN,NaN,NaN,NaN,5


In [18]:
d_filter.isnull().sum()

Sample Label(Number of cores for UCS, PLI, ITS)      0
UCS mean                                            42
PLI                                                 10
ITS  mean                                           15
SHV  (mean)                                         15
SHV (S)   (mean)                                   100
SV (m/s)    mean (B)Vp                             113
SV (m/s)    mean (B)Vs                             113
   SV (m/s)      mean (C)Vp                         10
   SV (m/s)      mean (C)Vs                         11
SDI (D) Id1 (%)                                      4
SDI (D) Id2 (%)                                      4
SDI (D) Id3 (%)                                      4
SDI (D) Id4 (%)                                      4
SDI (S) Id1 (%)                                    102
SDI (S) Id2 (%)                                    102
SDI (S) Id3 (%)                                    102
SDI (S) Id4 (%)                                    102
GAD       

In [19]:
d_filter.columns

Index(['Sample Label(Number of cores for UCS, PLI, ITS)', 'UCS mean', 'PLI ',
       'ITS  mean', 'SHV  (mean)', 'SHV (S)   (mean)',
       'SV (m/s)    mean (B)Vp', 'SV (m/s)    mean (B)Vs',
       '   SV (m/s)      mean (C)Vp', '   SV (m/s)      mean (C)Vs',
       'SDI (D) Id1 (%)', 'SDI (D) Id2 (%)', 'SDI (D) Id3 (%)',
       'SDI (D) Id4 (%)', 'SDI (S) Id1 (%)', 'SDI (S) Id2 (%)',
       'SDI (S) Id3 (%)', 'SDI (S) Id4 (%)', 'GAD'],
      dtype='object')

In [20]:
f = d_filter[['ITS  mean', 'GAD']]
f.head()

,ITS mean,GAD
0,5.77,2
1,NaN,5
2,2.36,5
3,1.83,5
4,2.42,5


In [21]:
f.shape

(150, 2)

### Physical properties

In [22]:
d10 = df.copy()

In [23]:
# Extracting the part before the hyphen
d10['GAD'] = d10['    Block     No'].str.split('-', n=1).str[0]

In [24]:
# Removing alphabets from the column
d10['GAD'] = d10['GAD'].str.replace('[^0-9]', '', regex=True)
d10.head()

,No,Block No,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,GAD
0,1,2,2.29,2.14,2.29,2287.0,2143.0,2191.0,2.09,6.57,14.8,0.17,2
1,2,5A,2.25,NaN,NaN,2253.0,NaN,NaN,2.10,8.37,NaN,NaN,5
2,3,5B,2.27,2.15,2.28,2269.0,2152.0,2279.0,2.12,7.37,12.7,0.15,5
3,4,5C,2.24,2.14,2.24,2242.0,2144.0,2243.0,2.12,5.99,9.8,0.11,5
4,5,5D-A,2.23,2.05,2.22,2233.0,2052.0,2218.0,2.06,8.93,16.6,0.20,5


In [25]:
d10.isnull().sum()

No                        0
    Block     No          0
 ?n  (gr/cm3)             5
   ?d     (gr/cm3)       12
 ?s       (gr/cm3)       12
 ?n           (kg/m3)     6
   ?d      (kg/m3)       12
     ?s      (kg/m3)     12
Gs                        6
WC     (%)                3
n           (%)          12
e                        12
GAD                       0
dtype: int64

In [26]:
# Assuming df is your DataFrame
d10.fillna(0, inplace=True)

In [27]:
d10.isnull().sum()

No                       0
    Block     No         0
 ?n  (gr/cm3)            0
   ?d     (gr/cm3)       0
 ?s       (gr/cm3)       0
 ?n           (kg/m3)    0
   ?d      (kg/m3)       0
     ?s      (kg/m3)     0
Gs                       0
WC     (%)               0
n           (%)          0
e                        0
GAD                      0
dtype: int64

In [28]:
# drop the unnecessary column
d10 = d10.drop(['No', '    Block     No'], axis=1)
d10.head()

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,GAD
0,2.29,2.14,2.29,2287.0,2143.0,2191.0,2.09,6.57,14.8,0.17,2
1,2.25,0.00,0.00,2253.0,0.0,0.0,2.10,8.37,0.0,0.00,5
2,2.27,2.15,2.28,2269.0,2152.0,2279.0,2.12,7.37,12.7,0.15,5
3,2.24,2.14,2.24,2242.0,2144.0,2243.0,2.12,5.99,9.8,0.11,5
4,2.23,2.05,2.22,2233.0,2052.0,2218.0,2.06,8.93,16.6,0.20,5


In [29]:
# Selecting columns for clustering
columns_for_clustering = [' ?n  (gr/cm3)', '   ?d     (gr/cm3)',
       ' ?s       (gr/cm3)', ' ?n           (kg/m3)', '   ?d      (kg/m3)',
       '     ?s      (kg/m3)', 'Gs ', 'WC     (%)', 'n           (%)']

In [30]:
# Selecting only the columns for clustering
data_for_clustering = d10[columns_for_clustering]

In [31]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Standardize the data using z-score normalization
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_clustering)
scaled_data

array([[ 0.31117985,  0.26060078,  0.34457778, ...,  0.0575638 ,
         0.44206667,  0.99043497],
       [ 0.21249702, -3.36544392, -3.38337391, ...,  0.08090047,
         1.30460463, -1.6354585 ],
       [ 0.26183844,  0.27754492,  0.32829851, ...,  0.12757382,
         0.82541687,  0.61784198],
       ...,
       [ 0.28650914,  0.24365665,  0.31201925, ...,  0.45428725,
         0.83500063,  0.83075226],
       [ 0.16315561,  0.34532145,  0.27946072, ...,  0.36094056,
        -1.0769585 , -0.52655078],
       [-0.10822217, -3.36544392, -3.38337391, ...,  0.38427723,
        -2.70619685, -1.6354585 ]])

In [32]:
# Convert scaled_data array back to DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=columns_for_clustering)
scaled_df.head()

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%)
0,0.311180,0.260601,0.344578,0.306904,0.267832,0.182047,0.057564,0.442067,0.990435
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458
2,0.261838,0.277545,0.328299,0.266203,0.283081,0.325245,0.127574,0.825417,0.617842
3,0.187826,0.260601,0.263181,0.205151,0.269526,0.266664,0.127574,0.164138,0.103309
4,0.163156,0.108104,0.230623,0.184800,0.113646,0.225983,-0.012446,1.572950,1.309800


In [33]:
concat_df = pd.concat([scaled_df, d10['e'], f], axis=1)
concat_df.head()

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean,GAD
0,0.311180,0.260601,0.344578,0.306904,0.267832,0.182047,0.057564,0.442067,0.990435,0.17,5.77,2
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00,NaN,5
2,0.261838,0.277545,0.328299,0.266203,0.283081,0.325245,0.127574,0.825417,0.617842,0.15,2.36,5
3,0.187826,0.260601,0.263181,0.205151,0.269526,0.266664,0.127574,0.164138,0.103309,0.11,1.83,5
4,0.163156,0.108104,0.230623,0.184800,0.113646,0.225983,-0.012446,1.572950,1.309800,0.20,2.42,5


In [34]:
s = concat_df.copy()

In [35]:
s = s.drop(['GAD', 'GAD'], axis=1)
s.head()

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean
0,0.311180,0.260601,0.344578,0.306904,0.267832,0.182047,0.057564,0.442067,0.990435,0.17,5.77
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00,NaN
2,0.261838,0.277545,0.328299,0.266203,0.283081,0.325245,0.127574,0.825417,0.617842,0.15,2.36
3,0.187826,0.260601,0.263181,0.205151,0.269526,0.266664,0.127574,0.164138,0.103309,0.11,1.83
4,0.163156,0.108104,0.230623,0.184800,0.113646,0.225983,-0.012446,1.572950,1.309800,0.20,2.42


In [36]:
# Assuming 'df' is your DataFrame
null_rows = s[s.isnull().any(axis=1)|(s == 'Nil').any(axis=1)]
null_rows

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00,NaN
8,-0.182234,-0.078282,0.035272,-0.140812,-0.076122,0.040475,0.220921,1.376483,1.274315,0.20,NaN
10,0.237168,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,1.635244,-1.635458,0.00,NaN
12,-0.058881,0.074215,0.067830,-0.023230,0.072981,0.061630,-0.012446,-0.185669,-0.074116,0.10,NaN
13,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.628950,-1.635458,0.00,NaN
14,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,-0.104207,-1.635458,0.00,NaN
37,0.311180,0.379210,0.328299,0.313688,0.384742,0.326873,-0.152466,-0.243172,-0.428967,0.07,Nil
45,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.044341,-1.635458,0.00,NaN
60,0.286509,-3.365444,-3.383374,0.288815,-3.363166,-3.383274,-0.105793,-0.113791,-1.635458,0.00,Nil
61,0.187826,0.396154,0.279461,0.211934,0.391520,0.286191,0.080900,-1.230299,-0.961243,0.04,Nil


In [37]:
# Assuming 'df' is your DataFrame
null_rows_index = s[s.isnull().any(axis=1)|(s == 'Nil').any(axis=1)].index
null_rows_index

Int64Index([1, 8, 10, 12, 13, 14, 37, 45, 60, 61, 70, 72, 75, 81, 95, 101, 116,
            119],
           dtype='int64')

In [38]:
rows_to_remove = [1, 8, 10, 12, 13, 14, 37, 45, 60, 61, 70, 72, 75, 81, 95, 101, 116,
            119]

In [39]:
# Assuming 'df' is your DataFrame
df_cleaned = s.drop(index=rows_to_remove)

# Now, df_cleaned does not contain rows with null values
df_cleaned

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean
0,0.311180,0.260601,0.344578,0.306904,0.267832,0.182047,0.057564,0.442067,0.990435,0.17,5.77
2,0.261838,0.277545,0.328299,0.266203,0.283081,0.325245,0.127574,0.825417,0.617842,0.15,2.36
3,0.187826,0.260601,0.263181,0.205151,0.269526,0.266664,0.127574,0.164138,0.103309,0.11,1.83
4,0.163156,0.108104,0.230623,0.184800,0.113646,0.225983,-0.012446,1.572950,1.309800,0.20,2.42
5,0.212497,0.175880,0.263181,0.234546,0.172948,0.266664,0.034227,0.811041,1.132375,0.19,2.5
...,...,...,...,...,...,...,...,...,...,...,...
145,0.286509,0.379210,0.344578,0.300121,0.391520,0.346400,0.500961,1.941924,-0.287027,0.08,2.97
146,0.237168,0.362266,0.312019,0.248113,0.364410,0.305718,0.407614,-0.430055,-0.437838,0.07,2.86
147,0.286509,0.243657,0.312019,0.300121,0.247500,0.312227,0.454287,0.835001,0.830752,0.16,2.49
148,0.163156,0.345321,0.279461,0.187061,0.342384,0.278055,0.360941,-1.076958,-0.526551,0.07,1.07


In [40]:
null_rows

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00,NaN
8,-0.182234,-0.078282,0.035272,-0.140812,-0.076122,0.040475,0.220921,1.376483,1.274315,0.20,NaN
10,0.237168,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,1.635244,-1.635458,0.00,NaN
12,-0.058881,0.074215,0.067830,-0.023230,0.072981,0.061630,-0.012446,-0.185669,-0.074116,0.10,NaN
13,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.628950,-1.635458,0.00,NaN
14,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,-0.104207,-1.635458,0.00,NaN
37,0.311180,0.379210,0.328299,0.313688,0.384742,0.326873,-0.152466,-0.243172,-0.428967,0.07,Nil
45,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.044341,-1.635458,0.00,NaN
60,0.286509,-3.365444,-3.383374,0.288815,-3.363166,-3.383274,-0.105793,-0.113791,-1.635458,0.00,Nil
61,0.187826,0.396154,0.279461,0.211934,0.391520,0.286191,0.080900,-1.230299,-0.961243,0.04,Nil


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [42]:
# Assuming your DataFrame is named 'df'
X = df_cleaned.drop(columns=['ITS  mean'])  # Features
y = df_cleaned['ITS  mean']  # Target variable

In [43]:
# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Create a Random Forest Regressor model


max_r2 = float('-inf')  # Initialize maximum R2 to negative infinity
best_random_state = None

for random_state in range(1, 101):
    # Create a Decision Tree Regressor model with a specific random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    lr_regressor = LinearRegression()

    # Fit the model to the training data
    lr_regressor.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = lr_regressor.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)

    # Check if current R2 is higher than the maximum
    if r2 > max_r2:
        max_r2 = r2
        best_random_state = random_state

print(f"Highest R-squared (R2): {max_r2} achieved with Random State: {best_random_state}")

Highest R-squared (R2): 0.5560509727991616 achieved with Random State: 40


In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Create a Random Forest Regressor model


max_r2 = float('-inf')  # Initialize maximum R2 to negative infinity
best_random_state = None

for random_state in range(1, 101):
    # Create a Decision Tree Regressor model with a specific random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    rf_regressor = RandomForestRegressor(random_state=random_state)

    # Fit the model to the training data
    rf_regressor.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = rf_regressor.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)

    # Check if current R2 is higher than the maximum
    if r2 > max_r2:
        max_r2 = r2
        best_random_state = random_state

print(f"Highest R-squared (R2): {max_r2} achieved with Random State: {best_random_state}")


Highest R-squared (R2): 0.6405112572034111 achieved with Random State: 58


In [46]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Assuming you have X_train, X_test, y_train, y_test defined

max_r2 = float('-inf')  # Initialize maximum R2 to negative infinity
best_random_state = None

for random_state in range(1, 101):
    # Create a Decision Tree Regressor model with a specific random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    regressor = DecisionTreeRegressor(random_state=random_state)

    # Fit the model to the training data
    regressor.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = regressor.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)

    # Check if current R2 is higher than the maximum
    if r2 > max_r2:
        max_r2 = r2
        best_random_state = random_state

print(f"Highest R-squared (R2): {max_r2} achieved with Random State: {best_random_state}")


Highest R-squared (R2): 0.6214480527492976 achieved with Random State: 67


In [48]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming you have X_train, X_test, y_train, y_test defined

max_r2 = float('-inf')  # Initialize maximum R2 to negative infinity
best_random_state = None

for random_state in range(1, 101):
    # Create an XGBoost Regressor model with a specific random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    xgb_regressor = XGBRegressor(random_state=random_state)

    # Fit the model to the training data
    xgb_regressor.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = xgb_regressor.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))


    # Check if current R2 is higher than the maximum
    if r2 > max_r2:
        max_r2 = r2
        best_random_state = random_state
        best_mae = mae  # Update the best MAE
        best_rmse = rmse  # Update the best RMSE

print(f"Highest R-squared (R2): {max_r2} achieved with Random State: {best_random_state}")
print(f"MAE for the best model: {best_mae}")
print(f"RMSE for the best model: {best_rmse}")

Highest R-squared (R2): 0.6473859233264969 achieved with Random State: 67
MAE for the best model: 0.61544264793396
RMSE for the best model: 0.9422957521285517


In [49]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
# Assuming you have X_train, X_test, y_train, y_test defined

max_r2 = float('-inf')  # Initialize maximum R2 to negative infinity
best_random_state = None
# Create a GradientBoostingRegressor
for random_state in range(1, 101):
    # Create an XGBoost Regressor model with a specific random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
    gregressor = GradientBoostingRegressor(n_estimators=10, learning_rate=0.27, random_state=random_state)

    # Fit the regressor to your training data
    gregressor.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = gregressor.predict(X_test)
    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Check if current R2 is higher than the maximum
    if r2 > max_r2:
        max_r2 = r2
        best_random_state = random_state
        best_model = gregressor  # Store the best model
        best_mae = mae  # Update the best MAE
        best_rmse = rmse  # Update the best RMSE

print(f"Highest R-squared (R2): {max_r2} achieved with Random State: {best_random_state}")
print(f"MAE for the best model: {best_mae}")
print(f"RMSE for the best model: {best_rmse}")



Highest R-squared (R2): 0.5828409382546651 achieved with Random State: 67
MAE for the best model: 0.7114757394853065
RMSE for the best model: 1.0249159492086677


### Model prediction

In [50]:
null_rows

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e,ITS mean
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00,NaN
8,-0.182234,-0.078282,0.035272,-0.140812,-0.076122,0.040475,0.220921,1.376483,1.274315,0.20,NaN
10,0.237168,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,1.635244,-1.635458,0.00,NaN
12,-0.058881,0.074215,0.067830,-0.023230,0.072981,0.061630,-0.012446,-0.185669,-0.074116,0.10,NaN
13,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.628950,-1.635458,0.00,NaN
14,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,-0.104207,-1.635458,0.00,NaN
37,0.311180,0.379210,0.328299,0.313688,0.384742,0.326873,-0.152466,-0.243172,-0.428967,0.07,Nil
45,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.044341,-1.635458,0.00,NaN
60,0.286509,-3.365444,-3.383374,0.288815,-3.363166,-3.383274,-0.105793,-0.113791,-1.635458,0.00,Nil
61,0.187826,0.396154,0.279461,0.211934,0.391520,0.286191,0.080900,-1.230299,-0.961243,0.04,Nil


In [51]:
X_test1 = null_rows.drop(['ITS  mean'], axis=1)
X_test1

,?n (gr/cm3),?d (gr/cm3),?s (gr/cm3),?n (kg/m3),?d (kg/m3),?s (kg/m3),Gs,WC (%),n (%),e
1,0.212497,-3.365444,-3.383374,0.230024,-3.363166,-3.383274,0.080900,1.304605,-1.635458,0.00
8,-0.182234,-0.078282,0.035272,-0.140812,-0.076122,0.040475,0.220921,1.376483,1.274315,0.20
10,0.237168,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,1.635244,-1.635458,0.00
12,-0.058881,0.074215,0.067830,-0.023230,0.072981,0.061630,-0.012446,-0.185669,-0.074116,0.10
13,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.628950,-1.635458,0.00
14,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,-0.104207,-1.635458,0.00
37,0.311180,0.379210,0.328299,0.313688,0.384742,0.326873,-0.152466,-0.243172,-0.428967,0.07
45,-5.338412,-3.365444,-3.383374,-4.864446,-3.363166,-3.383274,-4.819801,0.044341,-1.635458,0.00
60,0.286509,-3.365444,-3.383374,0.288815,-3.363166,-3.383274,-0.105793,-0.113791,-1.635458,0.00
61,0.187826,0.396154,0.279461,0.211934,0.391520,0.286191,0.080900,-1.230299,-0.961243,0.04


In [52]:
# Use the trained model to make predictions
predictions = xgb_regressor.predict(X_test1)

In [53]:
# Display the predictions
print("Predictions for the new data:")
print(predictions)

Predictions for the new data:
[2.1114259 1.2330396 3.2018378 1.1866565 1.2243857 1.2313713 3.0264046
 1.2310749 3.517598  2.134605  1.3764788 2.7730184 4.3086944 1.6430036
 1.0016621 1.2179626 1.2179626 3.550352 ]
